In [1]:
import numpy as np
import pandas as pd
import os
from random import choice, sample, randint
import matplotlib.pyplot as plt
import tqdm

In [2]:
import tensorflow as tf

from tensorflow.python.keras import backend
from tensorflow.keras import layers, Model

In [3]:
def dense_block(x, blocks, name):
    """
    A dense block
  
    Input:
        - x: input tensor.
        - blocks: integer, the number of building blocks.
        - name: string, block label.
      Output:
        - tensor for the block.
    """

    for i in range(blocks):
        x = conv_block(x, 32, name=name + '_block' + str(i + 1))
    return x

def transition_block(x, reduction, name):
    """
    A transition block
  
    Input:
        - x: input tensor.
        - reduction: float, compression rate at transition layers.
        - name: string, block label.
      Output:
        - tensor for the block.
    """
    
    bn_axis = 4 if backend.image_data_format() == 'channels_last' else 1
    x = layers.BatchNormalization(axis=bn_axis, epsilon=1.001e-5, name=name + '_bn')(x)
    x = layers.Activation('relu', name=name + '_relu')(x)
    x = layers.Conv3D(int(backend.int_shape(x)[bn_axis] * reduction), 1, use_bias=False, name=name + '_conv')(x)
    x = layers.AveragePooling3D(1, strides=(2,2,2), name=name + '_pool')(x)
    return x


def conv_block(x, growth_rate, name):
    """
    A building convolutional block for a dense block
  
    Input:
        - x: input tensor.
        - growth_rate: float, growth rate at dense layers.
        - name: string, block label.
      Output:
        - tensor for the block.
    """
    
    bn_axis = 4 if backend.image_data_format() == 'channels_last' else 1
    x1 = layers.BatchNormalization(axis=bn_axis, epsilon=1.001e-5, name=name + '_0_bn')(x)
    x1 = layers.Activation('relu', name=name + '_0_relu')(x1)
    x1 = layers.Conv3D(4 * growth_rate, 1, use_bias=False, name=name + '_1_conv')(x1)
    x1 = layers.BatchNormalization(axis=bn_axis, epsilon=1.001e-5, name=name + '_1_bn')(x1)
    x1 = layers.Activation('relu', name=name + '_1_relu')(x1)
    x1 = layers.Conv3D(growth_rate, 3, padding='same', use_bias=False, name=name + '_2_conv')(x1)
    x = layers.Concatenate(axis=bn_axis, name=name + '_concat')([x, x1])
    
    return x


In [4]:
class DensenetBuilder(object):
    @staticmethod
    def build(input_shape, num_outputs):
        print('original input shape:', input_shape)
        if len(input_shape) != 4:
            raise Exception("Input shape should be a tuple (nb_channels, kernel_dim1, kernel_dim2, kernel_dim3)")

        print('original input shape:', input_shape)
        # orignal input shape: (4,80,80,80)

        if backend.image_data_format() == 'tf':
            input_shape = (input_shape[1], input_shape[2], input_shape[3], input_shape[0])
        print('change input shape:', input_shape)

        input_0 = layers.Input(shape=input_shape)

        # 3D Convolution and pooling
        conv1 = layers.Conv3D(64, kernel_size=(3, 3, 3), strides=(1, 1, 1), padding='SAME', kernel_initializer='he_normal')(input_0)
        pool1 = layers.MaxPooling3D(pool_size=(3, 3, 3), strides=(2, 2, 2))(conv1)

        # Dense Block1
        x = dense_block(pool1, 6, name='conv1')
        x = transition_block(x, 0.5, name='pool1')
        x = dense_block(x, 6, name='conv2')
        x = transition_block(x, 0.5, name='pool2')
        x = dense_block(x, 6, name='conv3')
        print(x.shape)
        x = layers.GlobalAveragePooling3D(name='avg_pool')(x)
        print(x.shape)

        # Classifier block
        dense = layers.Dense(units=num_outputs, activation="softmax", kernel_initializer="he_normal")(x)

        model = Model(inputs=input_0, outputs=dense)
        return model

    @staticmethod
    def build_densenet(input_shape, num_outputs):
        # (4,80,80,80),20
        return DensenetBuilder.build(input_shape, num_outputs)

In [ ]:
def get_batches(X, Y, batch_size):
    n_samples = X.shape[0]
        
    # Shuffle at the start of epoch
    indices = np.arange(n_samples)
    np.random.shuffle(indices)
    
    for start in range(0, n_samples, batch_size):
        end = min(start + batch_size, n_samples)
        
        batch_idx = indices[start:end]
    
        yield X[batch_idx], Y[batch_idx] 

In [5]:
model = DensenetBuilder.build_densenet((4, 80, 80, 80), 20)
model.compile(loss="categorical_crossentropy", optimizer="sgd")
model.optimizer.learning_rate.assign(0.01)

original input shape: (4, 80, 80, 80)
original input shape: (4, 80, 80, 80)
change input shape: (4, 80, 80, 80)
(None, 1, 10, 10, 352)
(None, 352)


<tf.Variable 'UnreadVariable' shape=() dtype=float32, numpy=0.01>

In [5]:
ds = []

for file in list(os.listdir('./set')):
    ds.append(np.load('./set/' + file))

In [6]:
seq = []
for file in list(os.listdir('./SEQs')):
    seq.append(np.load('./SEQs/' + file))
    

In [7]:
train_test_split = 0.8

train_molecules = ds[:int(train_test_split*len(ds))]
test_molecules = ds[int(train_test_split*len(ds)):]

In [14]:
y_train = []
y_val = []

for i in range(len(train_molecules)):
    y_train.append(randint(0, 20))
    
for i in range(len(test_molecules)):
    y_val.append(randint(0, 20))    


In [15]:
history = model.fit(
    train_molecules,
    y_train,
    batch_size=5,
    epochs=2,
    # We pass some validation for
    # monitoring validation loss and metrics
    # at the end of each epoch
    validation_data=(test_molecules, y_val),
)

ValueError: Failed to find data adapter that can handle input: (<class 'list'> containing values of types {"<class 'numpy.ndarray'>"}), (<class 'list'> containing values of types {"<class 'int'>"})

In [ ]:
d

In [ ]:
N_train = 2
N_test = 4
rotations = False
N_epochs = 200

#  написать генератор батчей, энкодинг меток

In [ ]:
train_generator = BatchGenerator(train_molecules, batch_size=5, shuffle=True, rotation=rotations)
steps_per_epoch = len(train_generator)

val_generator = BatchGenerator(test_molecules, batch_size=N_test, shuffle=False, rotation=False)
val_steps = len(val_generator)